In [1]:
import pandas as pd
import numpy as np
import random

# Import data set and navigate

In [2]:
df = pd.read_excel("./algorel_fullDb_190223.xlsx")
df = df.loc[list(range(0,20000,100)),:]

# Advanced spacy

## Import Spacy

In [3]:
import spacy
nlp = spacy.load("fr_core_news_lg")

## Import my libraries and preprocess

In [4]:
import advanced_spacy as adv

In [5]:
cn = adv.text_normalizer.CompositeNumber()
us = adv.unit_system.UnitSystem()
us.load_measures()

## Spacy

### Tweak pipeline - Tokenizer and add functions

<p style='color:black; background-color:red'>
ATTENTION : unit lsit will need to not just be a \d followed by a unit
</p>

In [6]:
# nlp.tokenizer=custom_spacy.create_tokenizer(nlp)
nlp.tokenizer=spacy.tokenizer.registry.tokenizers.get("custom_tokenizerv1")()(nlp)
nlp.add_pipe("brand_unit_numbersv2",first=True)
nlp.add_pipe("preprocess",first=True)
nlp.add_pipe("false_positive_unit_cleaner",after="brand_unit_numbersv2")
nlp.remove_pipe("ner");


Custom Tokenizer instantiated: 442 words in vocab - 185 special chars


In [7]:
nlp.pipe_names

['preprocess',
 'brand_unit_numbersv2',
 'false_positive_unit_cleaner',
 'tok2vec',
 'morphologizer',
 'parser',
 'attribute_ruler',
 'lemmatizer']

## Visualize thingy

In [8]:
import advanced_spacy.custom_visualization as cu_vi

In [9]:
%%time
test = list(nlp.pipe(df.loc[:,"Texts"]))

CPU times: total: 15 s
Wall time: 30.9 s


In [10]:
from spacy.tokens import SpanGroup, Span
def detect_measure(doc,length):
    sg = SpanGroup(doc,name="measures",spans=[])
    for s in doc.sents:
        if len([t for t in s if not (t.is_punct or t.is_space)])<=length and any([t._.is_unit for t in s]):
            temp_span = Span(doc,s[0].i,s[-1].i+1,label='measure')
            sg.append(temp_span)
            temp_span = Span(doc,s[0].i,s[0].i+1,label='first')
            sg.append(temp_span)
            if s[-1].is_space:
                temp_span = Span(doc,s[-2].i,s[-2].i+1,label='last')
            else:
                temp_span = Span(doc,s[-1].i,s[-1].i+1,label='last')
            sg.append(temp_span)
    # attention
#     temp_span = Span(doc,0,1,label='measure')
#     sg.append(temp_span)
    #attention fin
    doc.spans['attributes'] = sg


In [150]:
%%time
for i in test:
    detect_measure(i,7)
# cu_vi.visualize_docspan(test,"attributes",["measure","first","last"])

CPU times: total: 15.6 ms
Wall time: 56.7 ms


In [151]:
from spacy.tokens import DocBin
hdd = DocBin(store_user_data=True, docs=test)
hdd.to_disk("./marked_data/50.spacy")

In [152]:
hdd2 = DocBin(store_user_data=True)
hdd2.from_disk("./marked_data/50.spacy")
data2 = list(hdd2.get_docs(nlp.vocab))
# cu_vi.visualize_docspan(data2,"attributes",["measure","first","last"])

In [153]:
cu_vi.visualize_docspan(data2,"attributes",["measure","first","last"])

Type 's' when you want to stop the visualizations


In [100]:
cu_vi.multi_displacy(test)

C:\Users\walra\AppData\Local\Programs\Python\Python311\Lib\site-packages\spacy\displacy\__init__.py:71: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


Type 's' when you want to stop the visualizations


# Test model

In [154]:
nlp2 = spacy.load("./Entrainement_test/bozon/model-last/")

Custom Tokenizer instantiated: 442 words in vocab - 185 special chars


In [155]:
nlp2.pipe_names

['tok2vec',
 'ner',
 'sentencizer',
 'spancat',
 'parser',
 'morphologizer',
 'tagger']

In [156]:
text_nlp2 = df.loc[0,"Texts"]

In [157]:
for i in ["parser","morphologizer","tagger"]:
    nlp2.disable_pipe(i)
print("pipe:",nlp2.pipe_names)
print("components:",nlp2.component_names)
print("disabeled",nlp2.disabled)

pipe: ['tok2vec', 'ner', 'sentencizer', 'spancat']
components: ['tok2vec', 'ner', 'sentencizer', 'spancat', 'parser', 'morphologizer', 'tagger']
disabeled ['parser', 'morphologizer', 'tagger']


In [158]:
for i in nlp2(text_nlp2).sents:
    print("START",i,"END")

START PROPFEU CREME LUSTRANTE boite 220 ml
www.gebstore.fr
821571
GEB
Chauffage - Climatisation/Cheminee/Entretien
CHALE
Description: Creme de protection et d'entretien des fontes et metaux ferreux. END
START 
Avantages: Sans odeur - Les exces sont lavables a l'eau savonneuse. END
START 
Utilisation: Donne un aspect patine aux elements en fonte et metaux ferreux des poeles, inserts et barbecues. END
START 
-_x0001_ Entretien et protection des fontes et metaux ferreux des cheminees et poeles (plaques, chenets, tuyaux, foyers). END
START 
Precautions: Appliquer au pinceau ou avec un chiffon sec - Laisser secher puis lustrer a la brosse ou au chiffon doux. END
START 
Couleur: noir
Conditionnement: boite 220ml END


In [163]:
for i in nlp2(text_nlp2):
    if(i.is_sent_start):
        print("\nSENTSTART")
    print(i,end=" ")


SENTSTART
PROPFEU CREME LUSTRANTE boite 220 ml 
 www.gebstore.fr 
 821571 
 GEB 
 Chauffage - Climatisation / Cheminee / Entretien 
 CHALE 
 Description : Creme de protection et d' entretien des fontes et metaux ferreux . 
SENTSTART

 Avantages : Sans odeur - Les exces sont lavables a l' eau savonneuse . 
SENTSTART

 Utilisation : Donne un aspect patine aux elements en fonte et metaux ferreux des poeles , inserts et barbecues . 
SENTSTART

 -_x0001 _ Entretien et protection des fontes et metaux ferreux des cheminees et poeles ( plaques , chenets , tuyaux , foyers ) . 
SENTSTART

 Precautions : Appliquer au pinceau ou avec un chiffon sec - Laisser secher puis lustrer a la brosse ou au chiffon doux . 
SENTSTART

 Couleur : noir 
 Conditionnement : boite 220 ml 

In [159]:
for i in nlp(text_nlp2).sents:
    print("START",i,"END")

START PROPFEU CREME LUSTRANTE boite 220 ml
 END
START www.gebstore.fr
 END
START 821571
 END
START GEB
 END
START Chauffage - Climatisation/Cheminee/Entretien
 END
START CHALE
 END
START Description: Creme de protection et d'entretien des fontes et metaux ferreux. END
START 
 END
START Avantages: Sans odeur - Les exces sont lavables a l'eau savonneuse. END
START 
 END
START Utilisation: Donne un aspect patine aux elements en fonte et metaux ferreux des poeles, inserts et barbecues. END
START 
 END
START -_x0001_ Entretien et protection des fontes et metaux ferreux des cheminees et poeles (plaques, chenets, tuyaux, foyers). END
START 
 END
START Precautions: Appliquer au pinceau ou avec un chiffon sec - Laisser secher puis lustrer a la brosse ou au chiffon doux. END
START 
 END
START Couleur: noir
 END
START Conditionnement: boite 220ml END


# Model to generate clean data

In [130]:
french_nlp = spacy.load("fr_core_news_lg")
with open("./Dummy_text.txt","r") as f:
    french_dummy_text = [i for i in f.readlines()]
french_dummy_text

['This is a simple test\n',
 'Is the suggester working correct ?\n',
 'This is unclear to me\n',
 '\n',
 'and this is a trick ']

In [131]:
from spacy.tokens import SpanGroup, Span
def detect_measure(doc,length):
    sg = SpanGroup(doc,name="measures",spans=[])
    temp_span = Span(doc,0,1,label='measure')
    sg.append(temp_span)
    doc.spans['attributes'] = sg


In [132]:
dummy_docs = list(nlp.pipe(french_dummy_text))
for i in dummy_docs:
    detect_measure(i,2)


In [133]:
cu_vi.visualize_docspan(dummy_docs,"attributes",["measure","first","last"])

Type 's' when you want to stop the visualization


In [135]:
from spacy.tokens import DocBin
hdd = DocBin(store_user_data=True, docs=dummy_docs)
hdd.to_disk("./marked_data/50.spacy")

In [138]:
hdd = DocBin(store_user_data=True)
hdd.from_disk("./marked_data/50.spacy")
dummy_docs = list(hdd.get_docs(french_nlp.vocab))


In [139]:
cu_vi.visualize_docspan(dummy_docs,"attributes",["measure","first","last"])

Type 's' when you want to stop the visualizations
